## Get School Funding Info from Chalkbeat

In [1]:
import requests
import time
import sys
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

Connect to chalkbeat funding information via google chrome driver

In [2]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get("https://www.chalkbeat.org/wp-content/static/shortcodes/searchfield/?sheet=publish&key=1A1tknyJWD20gu1qrBdvrzFY2yEhazOY5H-B_ImgMhSc&initialWidth=650&childId=1A1tknyJWD20gu1qrBdvrzFY2yEhazOY5H-B_ImgMhSc&parentTitle=Find%20out%20what%20your%20New%20York%20City%20school%20spends%20per%20student&parentUrl=https%3A%2F%2Fwww.chalkbeat.org%2Fposts%2Fny%2F2018%2F08%2F31%2Fhow-does-your-schools-budget-compare-to-others-in-new-york-city%2F")

Test connection to site and finding certain elements

In [3]:
form = driver.find_element_by_xpath("//input[contains(@class,'choices__input')]")
form.click()

In [4]:
school = driver.find_element_by_id("choices-select-item-choice-1000")
school.click()

In [5]:
fund_selector = '//td[contains(text(),"$")]'
funds = driver.find_element_by_xpath(fund_selector).text
funds

'$24,082'

In [6]:
exit_button = driver.find_element_by_xpath('//button[contains(@class,"choices__button")]')
exit_button.click()

In [11]:
school = '//th[contains(text(),"")]'
school_name = driver.find_element_by_xpath(school).text
school_name

Main function to go through school funding drop down and grab school and funding information

In [9]:
def get_the_funds():
    
    data = []
    
    for i in range(1,1628):
    
        form = driver.find_element_by_xpath("//input[contains(@class,'choices__input')]")
        form.click()

        school = driver.find_element_by_id("choices-select-item-choice-"+str(i))
        school.click()

        school_name = '//th[contains(text(),"")]'
        school_name_text = driver.find_element_by_xpath(school_name).text

        fund_selector = '//td[contains(text(),"$")]'
        fund_value = driver.find_element_by_xpath(fund_selector).text

        data.append((school_name_text,fund_value))

        exit_button = driver.find_element_by_xpath('//button[contains(@class,"choices__button")]')
        exit_button.click()
    
    return data

#vals = get_the_funds()
#vals[0:15]

In [10]:
vals[-1]

("Young Women's Leadership School, Queens", '$18,466')

Convert scraped values to datafram and check

In [12]:
import pandas as pd
df = pd.DataFrame(vals, columns=['School','$ per Student'])
df.head()

,School,$ per Student
0,47 The American Sign Language and English Seco...,"$29,476"
1,A. Philip Randolph Campus High School,"$20,440"
2,A.C.E. Academy for Scholars at the Geraldine F...,"$19,736"
3,Abraham Lincoln High School,"$20,884"
4,Abraham Lincoln YABC,"$12,248"


Save funding data for future use

In [13]:
df.to_pickle("funding_per_student.pkl")